<a href="https://colab.research.google.com/github/rderakhshan/AI-workforce/blob/main/Falcon_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Nov 23 21:36:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip
!pip install bitsandbytes==0.40.0
!pip install torch==2.1.0
!pip install transformers==4.30.0
!pip install accelerate==0.21.0
!pip install xformers==0.0.20
!pip install einops==0.6.1
!pip install langchain==0.0.233

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 117.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 105.4 MB/s eta 0:00:00
     ━━━━━

In [3]:
from google.colab import drive
work_dir = drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
!cd drive/

In [ ]:
ls

In [6]:
cd drive

/content/drive


In [7]:
cd MyDrive/

/content/drive/MyDrive


In [8]:
ls

 Agreement-1.pdf   'BME PROJECT'/      'Falcon 7B'      Thesis/
 AI/               'Colab Notebooks'/   Falcon-7B/      USB.zip
 Automation/        DOC004.PDF          Photos.zip      Utrecht.zip
 Automation.ipynb  'Document Copy'/     Residence.zip   Vahid_HostingAgreement_SignedScanned.pdf


In [9]:
cd AI/Falconchat

/content/drive/MyDrive/AI/Falconchat


In [10]:
ls

In [11]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.104s


In [12]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [13]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [14]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

model = AutoModelForCausalLM.from_pretrained( MODEL_NAME, trust_remote_code = True,\
                                             load_in_8bit = True, device_map = "auto"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Model device: {model.device}")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Model device: cuda:0


In [15]:
!nvidia-smi

Thu Nov 23 21:45:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   8153MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:

generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "transformers_version": "4.30.0",
  "use_cache": false
}

In [47]:
# model.config

In [17]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False


In [18]:

stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [19]:

generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'FalconForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNorm

In [20]:
# chain = ConversationChain(llm=llm)
# print(chain.prompt.template)

In [21]:

template = """
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [23]:
memory = ConversationBufferWindowMemory(
    memory_key = "history", k = 6, return_only_outputs = True)

chain = ConversationChain(llm = llm, memory = memory, prompt = prompt, verbose = True)

In [24]:
text = "How I can start to learn about large language models application? can you provide a roadmap in ten steps?"
res = chain.predict(input=text)
print(res)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:

> Finished chain.
 Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various h

In [25]:
text = "Does it requires programming skills?"
res = chain.predict(input=text)
print(res)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [26]:
text = "What about mathematical knowledge? Does it requres?"
res = chain.predict(input=text)
print(res)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [27]:
text = "Which programming language is suitable for piplining large language models?"
res = chain.predict(input=text)
print(res)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [28]:
text = "What couple of paragraphs about about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs)"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [29]:
text = " write two paragraphs about Understand the basics of natural language processing (NLP) and machine learning"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [30]:
text = "What is the main activity of banks?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can start to learn about large language models application? can you provide a roadmap in ten steps?
AI:  Sure! Here's a roadmap for learning about large language models application:

1. Understand the basics of natural language processing (NLP) and machine learning.
2. Learn about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs).
3. Familiarize yourself with popular frameworks and libraries used for building NLP applications, such as TensorFlow and PyTorch.
4. Practice building your own NLP applications using these frameworks and libraries.
5. Experiment with various hyperparameters to op

In [31]:
text = "How banks manges their portfolio through high interst rate periods?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Does it requires programming skills?
AI:  Yes, some basic knowledge of programming is required to implement NLP applications. However, there are many resources available online to help beginners learn programming skills.
User 
Human: What about mathematical knowledge? Does it requres?
AI:  Yes, some basic knowledge of mathematics is required to understand NLP concepts. This includes topics like linear algebra, calculus, and statistics.
User 
Human: Which programming language is suitable for piplining large language models?
AI:  Python and R are commonly used for NLP applications. Both languages have a wide range of libraries and tools that make them ideal for th

In [32]:
text = "What are the similiarities and differences of trading book and banking book portfolios in banks?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What about mathematical knowledge? Does it requres?
AI:  Yes, some basic knowledge of mathematics is required to understand NLP concepts. This includes topics like linear algebra, calculus, and statistics.
User 
Human: Which programming language is suitable for piplining large language models?
AI:  Python and R are commonly used for NLP applications. Both languages have a wide range of libraries and tools that make them ideal for this task.
User 
Human:
Human: What couple of paragraphs about about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs)
AI:   Recurrent Neural Networks (RNNs) are a type

In [33]:
text = "Can you summarize the type of models used in trading book and banking book for the capital requirement calculation?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Which programming language is suitable for piplining large language models?
AI:  Python and R are commonly used for NLP applications. Both languages have a wide range of libraries and tools that make them ideal for this task.
User 
Human:
Human: What couple of paragraphs about about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs)
AI:   Recurrent Neural Networks (RNNs) are a type of deep learning architecture that is often used for natural language processing tasks. They are designed to process sequential data, such as text or speech, and are particularly effective for tasks like language trans

In [34]:
text = "Probability of defauld, loss given default and exposure at default, are three main component in capital calculation. Write a paragraph summary about each of them."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What couple of paragraphs about about different types of language models, such as recurrent neural networks (RNNs) and convolutional neural networks (CNNs)
AI:   Recurrent Neural Networks (RNNs) are a type of deep learning architecture that is often used for natural language processing tasks. They are designed to process sequential data, such as text or speech, and are particularly effective for tasks like language translation and summarization. Convolutional Neural Networks (CNNs) are another type of deep learning architecture that is often used for natural language processing tasks. Unlike RNNs, CNNs are designed to process spatial data, such as images, and ar

In [35]:
text = "How banks do hedging against market risk?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human:  write two paragraphs about Understand the basics of natural language processing (NLP) and machine learning
AI:  Natural Language Processing (NLP) is a branch of Artificial Intelligence (AI) that focuses on understanding and interpreting human language. It involves analyzing and recognizing patterns in language, which can be used to perform tasks such as sentiment analysis, information extraction, and dialogue systems. Machine Learning is a subset of AI that involves algorithms that can learn from data and improve their performance over time. It is used to build predictive models that can be used to make predictions about future events. In order to understand NL

In [36]:
text = "What are the most important models for credit acceptance scorecard which banks use for their internal rating purposes?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the main activity of banks?
AI:  Banks typically handle financial transactions, such as accepting deposits and withdrawals, issuing loans, and providing investment services. They also offer additional services such as credit cards, debit cards, and checking accounts.
User 
Human:
Human: How banks manges their portfolio through high interst rate periods?
AI:   Banks typically manage their portfolio by investing in assets that generate higher returns than the cost of borrowing money. These assets include government bonds, stocks, and other investments that provide long-term value to the bank. During high interest rate periods, banks may invest in short-ter

In [37]:
text = "Does logistic regression is used in credit ecoring models?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How banks manges their portfolio through high interst rate periods?
AI:   Banks typically manage their portfolio by investing in assets that generate higher returns than the cost of borrowing money. These assets include government bonds, stocks, and other investments that provide long-term value to the bank. During high interest rate periods, banks may invest in short-term assets such as Treasury bills or commercial paper to generate higher returns. Additionally, banks may use derivatives such as futures or options to hedge against risk and lock in profits.
User 
Human:
Human: What are the similiarities and differences of trading book and banking book portfolios

In [38]:
text = "Does banks use survival analysis for predicting probability of default?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are the similiarities and differences of trading book and banking book portfolios in banks?
AI:   The trading book and banking book portfolios in banks are similar in that they both involve managing assets and liabilities. However, the trading book is focused on generating income from trading securities, while the banking book is focused on lending money and managing customer deposits. Additionally, the trading book is usually more volatile than the banking book due to the nature of the assets involved.
User 
Human:
Human: Can you summarize the type of models used in trading book and banking book for the capital requirement calculation?
AI:    In the bankin

In [39]:
text = "What is the main responsibilities of basell comittee on banking?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Can you summarize the type of models used in trading book and banking book for the capital requirement calculation?
AI:    In the banking industry, the capital requirement calculation is typically based on Basel III regulations, which require banks to maintain a certain level of equity capital and liquidity ratios. This is done by calculating the Tier 1 Capital ratio, which is the amount of equity capital required to support the bank's total assets. For the trading book, the capital requirement calculation is typically based on market risks and volatility, which can be modeled using Monte Carlo simulations or historical data analysis.
User 
Human: Probability of

In [41]:
text = "summarize the main responsibilites of Basel committee in five seperate paragraphs."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How banks do hedging against market risk?
AI:   Banks can hedge against market risk by taking positions in the foreign exchange market, buying and selling derivatives, and engaging in forward contracts. Foreign exchange markets allow banks to buy and sell currencies to protect themselves from fluctuations in currency values. Derivatives are financial instruments that derive their value from underlying assets such as stocks, bonds, or commodities. Forward contracts are agreements between two parties to buy and sell assets at predetermined prices and dates. By utilizing these tools, banks can reduce their exposure to market risk and increase their profitability.
U

In [44]:
text = "What are those 5 main factors which banks has to consider in bulding model taxonomy?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Does banks use survival analysis for predicting probability of default?
AI:     Yes, survival analysis is often used in the banking industry to predict the likelihood of a borrower defaulting on a loan. It is a common method used to analyze time series data and can be used to identify patterns in loan delinquency rates. Survival analysis takes into account variables such as the length of time since the last payment, the number of payments made, and other factors to determine the likelihood of default. The results of the analysis can be used to develop predictive models that can accurately identify high-risk borrowers and reject low-risk applicants.
User 
Human: 

In [43]:
text = "How europian central bank supervises the banks in euro zone?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Does logistic regression is used in credit ecoring models?
AI:    Yes, logistic regression is often used in credit scoring models to predict the likelihood of a borrower paying back a loan. It is a commonly used technique in the finance industry to build predictive models that can accurately identify high-risk borrowers and reject low-risk applicants. The model uses a combination of features such as age, income, occupation, and education level to generate a prediction score that can be used to determine the risk of defaulting on a loan.
User 
Human:
Human: Does banks use survival analysis for predicting probability of default?
AI:     Yes, survival analysis is o

In [45]:
text = "Write precise and consise email to my manage and ask her to attend our next quarterly meeting. Note that, her name is Melike."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the main responsibilities of basell comittee on banking?
AI:     The Basel Committee on Banking Supervision is responsible for developing guidelines and standards for the global banking industry. Its main responsibilities include ensuring that banks maintain adequate levels of capital, assessing the soundness of individual institutions, and monitoring systemic risks in the banking sector. The committee also sets out rules and requirements for banks to follow when they issue new products and services, including consumer and business accounts. Additionally, the committee works to ensure that banks comply with international regulatory standards and best pra

In [46]:
text = "How I can travel from Rotterdam to New York? what are the possibilities?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Write five pargraphs to describe the main responsibilites of Basel committee
AI:      The Basel Committee on Banking Supervision is responsible for developing guidelines and standards for the global banking industry. Its main responsibilities include ensuring that banks maintain adequate levels of capital, assessing the soundness of individual institutions, and monitoring systemic risks in the banking sector. The committee also sets out rules and requirements for banks to follow when they issue new products and services, including consumer and business accounts. Additionally, the committee works to ensure that banks comply with international regulatory standards

In [47]:
text = "What are unique features of Dutch which other languages do not have them?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: summarize the main responsibilites of Basel committee in five seperate paragraphs.
AI:       The Basel Committee on Banking Supervision is responsible for developing guidelines and standards for the global banking industry. Its main responsibilities include ensuring that banks maintain adequate levels of capital, assessing the soundness of individual institutions, and monitoring systemic risks in the banking sector. The committee also sets out rules and requirements for banks to follow when they issue new products and services, including consumer and business accounts. Additionally, the committee works to ensure that banks comply with international regulatory st

In [48]:
text = "What are unique features of Dutch languege?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are those main factors which banks has to consider in bulding model taxonomy? write them in 10 bullet points.
AI:      The main factors that banks have to consider in building a model taxonomy include: 
- Identifying the purpose of the model
- Determining the scope of the model
- Defining the data elements and sources
- Establishing the structure of the model
- Developing the data validation process
- Testing the model
- Documenting the model
- Implementing the model
- Updating the model
- Maintaining the model
User 
Human:
Human: How europian central bank supervises the banks in euro zone?
AI:       The European Central Bank (ECB) supervises the banks in t

In [44]:
text = "What are the differencess of acustic violin and electric violin from playing perspective?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How europian central bank supervises the banks in euro zone?
AI:    The European Central Bank (ECB) supervises the banks in the Eurozone through various tools such as stress tests, regular inspections, and monitoring of banks’ balance sheets. The ECB also implements policies such as quantitative easing to stimulate economic growth and maintain price stability. In addition, the ECB has implemented new rules to increase transparency and accountability in the banking sector.
User 
Human:
Human: Write precise and consise email to my manage and ask her to attend our QBR meeting. Note that, her name is Melike.
AI:      Dear Melike, 

I am writing to invite you to atte

In [49]:
text = "Does electric violin requires additional equipments to use? if so, what are them?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How europian central bank supervises the banks in euro zone?
AI:       The European Central Bank (ECB) supervises the banks in the Eurozone by setting minimum requirements for banks to meet in order to remain eligible for ECB funding. These requirements include having sufficient capital, maintaining adequate liquidity ratios, and complying with regulations such as anti-money laundering and counterfeiting laws. In addition, the ECB regularly checks the financial health of banks and may impose restrictions or penalties if necessary.
User 
Human:
Human: What are those 5 main factors which banks has to consider in bulding model taxonomy?
AI:       The main factors t

In [50]:
text = "Write one page summary about, geography of netherlands"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are those 5 main factors which banks has to consider in bulding model taxonomy?
AI:       The main factors that banks have to consider in building a model taxonomy include: 
- Identifying the purpose of the model
- Determining the scope of the model
- Defining the data elements and sources
- Establishing the structure of the model
- Developing the data validation process
- Testing the model
- Documenting the model
- Implementing the model
- Updating the model
- Maintaining the model
User 
Human:
Human: Write precise and consise email to my manage and ask her to attend our next quarterly meeting. Note that, her name is Melike.
AI:        Dear Melike, 

I am 

In [51]:
text = "Write one page summary of history of Netherlands."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Write precise and consise email to my manage and ask her to attend our next quarterly meeting. Note that, her name is Melike.
AI:        Dear Melike, 

I am writing to request your attendance at our upcoming quarterly meeting. This important event will take place on October 15th, 2021, at 2pm. Your presence is essential to help us move forward with our goals and objectives. 

I look forward to seeing you there. 

Best regards, 
[Your Name]
User 
Human: How I can travel from Rotterdam to New York? what are the possibilities?
AI:        There are several options available for traveling from Rotterdam to New York, including flights, trains, and buses. Flights are o

In [52]:
text = "Why Utrecht is special for Dutch community?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How I can travel from Rotterdam to New York? what are the possibilities?
AI:        There are several options available for traveling from Rotterdam to New York, including flights, trains, and buses. Flights are offered by various airlines, including KLM Royal Dutch Airlines, Air France, and Delta. Train tickets can be purchased through the Netherlands Railways website. Buses can be booked through websites like FlixBus or Greyhound. It is recommended to book in advance to secure the best prices and availability.
User 
Human: What are unique features of Dutch which other languages do not have them?
AI:        One unique feature of Dutch is the use of the word "Ye

In [53]:
text = "My friend's name is Magda, write a poem for her birthday."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are unique features of Dutch which other languages do not have them?
AI:        One unique feature of Dutch is the use of the word "Yes" to agree with a statement. For example, instead of saying "Yes", a Dutch person would say "Ja". Another feature is the use of bicycles as a primary mode of transportation in cities. In the Netherlands, cycling is seen as a healthy and sustainable way to get around. Additionally, Dutch people often eat their meals later in the day, typically between 12pm and 2pm. This is due to the fact that lunch is considered the biggest meal of the day in the Netherlands.
User 
Human: What are unique features of Dutch languege?
AI:      

In [ ]:
# !streamlit run chatbot_app.py & npx localtunnel --port 8501

In [ ]:
!nvidia-smi

Sat Jul 15 13:09:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |   8351MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "transformers_version": "4.30.0",
  "use_cache": false
}

In [ ]:
model.config

RWConfig {
  "_name_or_path": "tiiuae/falcon-7b-instruct",
  "alibi": false,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "RWForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "tiiuae/falcon-7b-instruct--configuration_RW.RWConfig",
    "AutoModelForCausalLM": "tiiuae/falcon-7b-instruct--modelling_RW.RWForCausalLM"
  },
  "bias": false,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "hidden_dropout": 0.0,
  "hidden_size": 4544,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "RefinedWebModel",
  "multi_query": true,
  "n_head": 71,
  "n_layer": 32,
  "parallel_attn": true,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_i

In [ ]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Dwight K Schrute?
AI:
""".strip()
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

In [ ]:
%%time

with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

CPU times: user 7.56 s, sys: 0 ns, total: 7.56 s
Wall time: 7.55 s


In [ ]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Dwight K Schrute?
AI: Dwight K Schrute is a fictional character in the American television series "The Office". He is portrayed by actor Rainn Wilson and appears to be highly intelligent, but socially awkward and often misinterprets social cues.
User 


## Stop LLM From Rambling

In [ ]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

In [ ]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [ ]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [ ]:
res = llm(prompt)
print(res)

 Dwight K Schrute is a fictional character in the American television series "The Office". He is portrayed by actor Rainn Wilson and appears to be highly intelligent, but socially awkward and often misinterprets social cues.
User 


## Conversation Chain

In [ ]:
chain = ConversationChain(llm=llm)
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


### Custom Prompt

In [ ]:
template = """
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [ ]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.
 SchruteAuto
User 


### Cleaning Output

In [ ]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)

## Try It Out

In [ ]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain(text)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.


In [ ]:
res.keys()

dict_keys(['input', 'history', 'response'])

In [ ]:
print(res["response"])

SchruteAuto


In [ ]:
text = "Think of a slogan for the company"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: SchruteAuto
Human: Think of a slogan for the company
AI:

> Finished chain.
Drive Big With SchruteAuto


In [ ]:
text = "Choose a domain name for the company"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: SchruteAuto
Human: Think of a slogan for the company
AI: Drive Big With SchruteAuto
Human: Choose a domain name for the company
AI:

> Finished chain.
schruteauto.com


In [ ]:
text = "Write a tweet that introduces the company and introduces the first car built by the company"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: SchruteAuto
Human: Think of a slogan for the company
AI: Drive Big With SchruteAuto
Human: Choose a domain name for the company
AI: schruteauto.com
Human: Write a tweet that introduces the company and introduces the first car built by the company
AI:

> Finished chain.
Introducing SchruteAuto! We build powerful family cars with big V8 engines. Check out our 

In [ ]:
text = "Write a short marketing email to sell the first car from the company - 700HP family sedan from a supercharged V8 with manual gearbox."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office. 
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical. 
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: SchruteAuto
Human: Think of a slogan for the company
AI: Drive Big With SchruteAuto
Human: Choose a domain name for the company
AI: schruteauto.com
Human: Write a tweet that introduces the company and introduces the first car built by the company
AI: Introducing SchruteAuto! We build powerful family cars with big V8 engines. Check out our website for more in